# Features engineering

In [2]:
# Libraries and parameters
import pandas as pd
pd.set_option('display.max_columns', None)
import math

# Functions


In [ ]:
# Import data
data = pd.read_csv("../raw_data/players.csv")

In [3]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between the home team and the away team
    => return the distance in kilometer
    """
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    radius = 6371.0

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = radius * c

    return distance

In [ ]:
# Calculate distance between home and away team
data["distance"] = data.apply(lambda row: haversine(row["stadium_lat"], row["stadium_lon"], row["away_stadium_lat"], row["away_stadium_lon"]), axis=1)

In [ ]:
# Calculate stats of bater before he bats
pd.to_datetime()


data["hitter_previous_stats"] = data.apply((lambda row: data.iloc[0:row.name][data["hitter_id"] == row["hitter_id"]]["y_target"].mean()), axis=1)

In [ ]:
# Calculate stats of pitcher before he pitchs


data["pitcher_previous_stats"] = data.apply((lambda row: data.iloc[0:row.name][data["pitcher_id"] == row["pitcher_id"]]["y_target"].mean()), axis=1)